# Data Mining (CpSc 8650) Course Project  Quality Evaluation of Skull Stripped Brain MRI Images

In [1]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import nibabel as nib
import random
import csv
from scipy import ndimage
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
# !conda create -n my_environment -y
# !source activate my_environment
# !pip install sklearn
# !pip install nibabel
#!wget --no-check-certificate https://dyslexia.computing.clemson.edu/BET_BSE/BET_BSE_DATA.zip

In [20]:
# load and return data 
def load_data(csv_file_path, test_size = 0.3, x_names = [], y_names = []):

    
    """
    Load training data and split it into training and validation set
    """
    print ("Loading data from: ",csv_file_path)
    #reads CSV file into a single dataframe variable
    data_df = pd.read_csv(csv_file_path, names=x_names+y_names, skiprows=1)

    #yay dataframes, we can select rows and columns by their names
    X = data_df[x_names].values
    #and our steering commands as our output data
    y = data_df[y_names ].values

    #now we can split the data into a training (80), testing(20), and validation set
    #thanks scikit learn
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=0)

    return X_train, X_valid, y_train, y_valid

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    print((width_factor, height_factor, depth_factor))
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    print(img.shape)
    return img

def process_scan(path):
#     global i
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    #Resize width, height and depth
    volume = resize_volume(volume)
#     i+=1
#     print(i)
    return volume

In [21]:
@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [0]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
#     volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label



In [22]:
label_file_path = os.path.join(os.path.abspath(os.pardir), "dataset", "Label_file.csv")
X_train, X_valid, y_train, y_valid = load_data(label_file_path, x_names = ["Filename"], y_names = ["Recognizable-Facial-Feature","Brain-Feature-Loss"])

Loading data from:  /home/abastol/DataMining/Project/dataset/Label_file.csv


In [23]:
y_train[y_train=='Yes'] = 1
y_train[y_train=='No'] = 0
y_valid[y_valid=='Yes'] = 1
y_valid[y_valid=='No'] = 0

#### Process Training Data and Testing Data

In [24]:
os.path.join(os.path.abspath(os.pardir), "dataset", "files", )

'/home/abastol/DataMining/Project/dataset/files'

In [25]:
X_train_data = np.array([process_scan(os.path.join(os.path.abspath(os.pardir), "dataset", "files", path[0]+".gz")) for path in X_train])
X_valid_data = np.array([process_scan(os.path.join(os.path.abspath(os.pardir), "dataset", "files", path[0]+".gz")) for path in X_valid])

(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4383561643835616)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4383561643835616)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(128, 128, 64)
(0.5, 0.5, 0.4266666666666667)
(12

In [27]:
X_train_data.shape

(917, 128, 128, 64)

#### Assign 1,0

#### Split data 80-20

In [28]:
print(
    "Number of samples in train and validation are %d and %d."
    % (X_train_data.shape[0], X_valid_data.shape[0])
)

Number of samples in train and validation are 917 and 394.


In [29]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((X_train_data, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((X_valid_data, y_valid))

batch_size = 24
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(X_train_data))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(X_valid_data))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

2022-04-06 10:47:38.541264: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-06 10:47:39.276696: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38397 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:3b:00.0, compute capability: 8.0
2022-04-06 10:47:39.277994: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38397 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:d8:00.0, compute capability: 8.0


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

## Displaying Image

In [ ]:
image = X_train_data.take(1)
print("Dimension of the CT scan is:", image.shape)
plt.imshow(np.squeeze(image[:, :, 30]), cmap="gray")
plt.show()

In [ ]:
def plot_slices(num_rows, num_columns, width, height, data):
    """Plot a montage of 20 CT slices"""
    data = np.rot90(np.array(data))
    data = np.transpose(data)
    data = np.reshape(data, (num_rows, num_columns, width, height))
    rows_data, columns_data = data.shape[0], data.shape[1]
    heights = [slc[0].shape[0] for slc in data]
    widths = [slc.shape[1] for slc in data[0]]
    fig_width = 12.0
    fig_height = fig_width * sum(heights) / sum(widths)
    f, axarr = plt.subplots(
        rows_data,
        columns_data,
        figsize=(fig_width, fig_height),
        gridspec_kw={"height_ratios": heights},
    )
    for i in range(rows_data):
        for j in range(columns_data):
            axarr[i, j].imshow(data[i][j], cmap="gray")
            axarr[i, j].axis("off")
    plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    plt.show()


# Visualize montage of slices.
# 4 rows and 10 columns for 100 slices of the CT scan.
plot_slices(4, 10, 128, 128, image[:, :, :40])

In [ ]:
def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=16, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=32, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=2, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=128, height=128, depth=64)
model.summary()

In [ ]:
# Compile model.
initial_learning_X_train0.00001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification_relu.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 100
model.fit(
    X_train_data,
    validation_data=X_valid_data,
    epochs=epochs,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(20, 3))
# ax = ax.ravel()

# for i, metric in enumerate(["acc", "loss"]):
#     ax[i].plot(model.history.history[metric])
#     ax[i].plot(model.history.history["val_" + metric])
#     ax[i].set_title("Model {}".format(metric))
#     ax[i].set_xlabel("epochs")
#     ax[i].set_ylabel(metric)
#     ax[i].legend(["train", "val"])

### Predict with a image

In [ ]:
# file = process_scan('Yes/IXI002-Guys-0828-T1_bse_less_s5_r1.nii.gz')
# file = tf.expand_dims(file, axis=0)

# prediction = model.predict(file)


# yes,no = prediction[0] * 100, (1-prediction[0]) *100

# print("Yes=",yes[0],"No=",no[0])
# if(yes[0]>50):
#     print("Facial Feature Recognizable")
# else:
#     print("Facial Feature not recognizable")